# SIMULATION

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Run</a></span></li><li><span><a href="#Win-Rate" data-toc-modified-id="Win-Rate-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Win Rate</a></span></li><li><span><a href="#Q-Values" data-toc-modified-id="Q-Values-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Q Values</a></span></li><li><span><a href="#Visits" data-toc-modified-id="Visits-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Visits</a></span></li><li><span><a href="#Extraction" data-toc-modified-id="Extraction-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Extraction</a></span></li></ul></div>

### Libraries
***

In [2]:
# Custom libraries
import environment as uno

# Public libraries
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def bold(string):
    chr_start = "\033[1m"
    chr_end = "\033[0m"
    print (chr_start + string + chr_end)
    
def underline(string):
    chr_start = "\033[4m"
    chr_end = "\033[0m"
    print(chr_start + string + chr_end)

### Parameters
***

In [4]:
# Widgets for settings
widg_sim     = widgets.IntText(value = 100, description = "Simulations:")
widg_algo    = widgets.Dropdown(options=["Monte-Carlo","Q-Learning"], value="Monte-Carlo", description="Algorithm:")
widg_q       = widgets.Dropdown(options=[True,False], value=True, description="Q-Table:")
widg_comment = widgets.Dropdown(options=[True,False], value=False, description="Show Game:")

# Widgets for parameters
widg_epsilon = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description="Epsilson:", readout_format=".2f")
widg_step    = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description="Step Size:", readout_format=".2f")
widg_gamma   = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description="Gamma:", readout_format=".2f")

underline ("General Settings:")
display(widg_sim)
display(widg_algo)
display(widg_q)
display(widg_comment)

underline ("\nModel Parameters:")
display(widg_epsilon)
display(widg_step)
display(widg_gamma)

General Settings:


IntText(value=100, description='Simulations:')

Dropdown(description='Algorithm:', options=('Monte-Carlo', 'Q-Learning'), value='Monte-Carlo')

Dropdown(description='Q-Table:', options=(True, False), value=True)

Dropdown(description='Show Game:', index=1, options=(True, False), value=False)


Model Parameters:


FloatSlider(value=0.2, description='Epsilson:', max=1.0, step=0.01)

FloatSlider(value=0.2, description='Step Size:', max=1.0, step=0.01)

FloatSlider(value=0.2, description='Gamma:', max=1.0, step=0.01)

In [ ]:
# Agent parameters
agent_info = {"epsilon"  : widg_epsilon.value, 
              "step_size": widg_step.value, 
              "gamma"    : widg_gamma.value,
              "q_exist"  : widg_q.value}

agent_info

### Run
***

In [ ]:
winners, turns, visits = list(), list(), list()

In [ ]:
run = uno.tournament(iterations = widg_sim.value,
                     algo       = widg_algo.value,
                     comment    = widg_comment.value,
                     agent_info = agent_info)

winners.extend(run[0])
turns.extend(run[1])
visits.extend(run[2])

### Win Rate
***

In [ ]:
winners.count("Bernhard")/len(winners)

In [ ]:
fig = plt.figure(1, figsize=(15,7))

plt.hist(turns, bins =100, edgecolor = "w")
plt.title("Turns Distribution")
plt.xlabel("Turns per Game")
plt.ylabel("Games")

plt.show()

In [ ]:
result = pd.concat([pd.Series(winners), pd.Series(turns)], axis = 1)
result = result.rename(columns={0:"Winner",1:"Turns"})

result["Win_rate"] = result["Winner"].apply(lambda x: 1 if x == "Bernhard" else 0)
result["Win_rate"] = result["Win_rate"].cumsum()/(result.index+1)

result.tail()

In [ ]:
result["Winner"][-100000:].value_counts()

In [ ]:
fig = plt.figure(1, figsize=(15,7))

plt.plot(result.index, result["Win_rate"])
plt.hlines(0.5, 0, len(winners), colors="grey", linestyles="dashed")

plt.title("Learning Curve")
plt.xlabel("Simulations")
plt.ylabel("Win Rate")
plt.ylim((0,1))

plt.show()

### Q Values
***

In [ ]:
q_table = pd.DataFrame(uno.agent.q)
q_table["IDX"] = q_table.index
q_table = q_table.set_index("IDX", drop = True)
q_table.index.name = None

In [ ]:
q_pivot = q_table.copy()

q_pivot["HAND"] = q_pivot.index.map(lambda x: sum(x[1:10]))
q_pivot = q_pivot.replace(0, np.NaN)
q_pivot = q_pivot.groupby("HAND").mean()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15,10))

sns.heatmap(q_pivot, 
            fmt=".2f", 
            cmap = "OrRd", 
            linewidth = 0.3, 
            annot = True,
            cbar = False,
            center = 0.1)

ax.title.set_text("Q Values")
ax.set(xlabel="Action taken", ylabel="Hand Cards")
plt.show()

### Visits
***

In [ ]:
q_visited = (q_table != 0).values.sum()
q_possible = 0

for i in range(len(q_table)):
    q_possible += sum(q_table.index[i][8:])
    
print (f'From {q_possible:,} possible combinations {q_visited:,} or {q_visited/q_possible*100:.2f}% have been visited')

In [ ]:
fig = plt.figure(1, figsize=(15,7))
plt.plot(q_visited)
plt.show()

In [ ]:
q_visit = uno.agent.q_visit.copy()

q_visit["HAND"] = q_visit.index.map(lambda x: sum(x[1:10]))
q_visit = q_visit.replace(0, np.NaN)
q_visit = q_visit.groupby("HAND").sum()

In [ ]:
fig = plt.figure(1, figsize=(15,7))

dist_hand = q_visit.sum(axis = 1)

plt.bar(dist_hand.index, dist_hand.values)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (15,10))

sns.heatmap(q_visit, 
            fmt=",.0f", 
            cmap = "OrRd", 
            linewidth = 0.3, 
            annot = True,
            cbar = False,
            center = 0.1)

ax.title.set_text("Visits")
ax.set(xlabel="Action taken", ylabel="Hand Cards")
plt.show()

### Extraction
***

In [ ]:
q_table.to_csv("02 Extracts/q_table.csv" , sep = ";")
uno.agent.q_visit.to_csv("02 Extracts/q_visit.csv" , sep = ";")